In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tkinter import Tk
from tkinter import filedialog as fd
import os
import math

In [5]:
#User defined constants for the program
crystalline_layers = 0.0
amorphous_layers = 3.0
totalLayers = crystalline_layers + amorphous_layers
bond_region = "amorphous"
bond_cutoff = 2.58 #angstroms
x_boundary = "periodic"
y_boundary = "periodic"
z_boundary = "periodic"
coords_scaled = False

In [8]:
bondLengths = []
bondAngles = []
danglingBonds = 0
floatingBonds = 0
strainedBonds = 0

In [12]:
#Load tkinter and read in the user chosen file
bondLengths = []
bondAngles = []
danglingBonds = 0
floatingBonds = 0
strainedBonds = 0
root = Tk()
folder = fd.askdirectory(title = "Choose the folder containing the optimized a-Si structures")
root.withdraw()
nfiles = 0
files = os.listdir(folder)
for file in files:
    iterator = 0
    nAtoms = 0
    xSize = 0
    ySize = 0
    zSize = 0
    zBoxSize = 0
    z_lower_boundary = 0.0
    z_upper_boundary = 0.0
    atomicInformationDict = {}
    atomicLocations = []
    print(file)
    filename = folder + "/" + file
    nfiles += 1
    f = open(filename, 'r')
    iterator = 0
    for line in f:
        #if iterator == 3:
        #    nAtoms = float(line)
        if iterator == 3:
            newline = line.split()
            xSize = float(newline[1]) - float(newline[0])
        if iterator == 4:
            newline = line.split()
            ySize = float(newline[1]) - float(newline[0])
        if iterator == 5:
            newline = line.split()
            zSize = float(newline[1]) - float(newline[0])

            #Set z_boundaries to be those of the amorphous layer
            zl_bonded = (crystalline_layers + 0.5)*zSize/totalLayers
            zu_bonded = (totalLayers - 0.5)*zSize/totalLayers

            zl_comparison = crystalline_layers*zSize/totalLayers
            zu_comparison = zSize

        elif iterator >= 7:
            newline = line.split()
            floatline = [float(i) for n, i in enumerate(newline) if n>0]
            identifier = iterator - 6
            atomicInformationDict[identifier] = floatline
            if coords_scaled:
                if zl_comparison <= floatline[4]*zSize and floatline[4]*zSize<=zu_comparison:
                    unscaledCoordinates = [floatline[2]*xSize, floatline[3]*ySize, floatline[4]*zSize]
                    atomicLocations.append(unscaledCoordinates)
                    nAtoms += 1
            else:
                if zl_comparison <= floatline[2] and floatline[2] <= zu_comparison:
                    atomicLocations.append(floatline)
                    nAtoms += 1
        iterator += 1

    #make an array of all the atoms
    atomicArray = np.array(atomicLocations)

    #create dictionary of target atoms and those who they are bonded to
    bondDictionary = {}
    for n, i in enumerate(atomicArray[:-1]):
        if zl_bonded <= i[2] and i[2] <= zu_bonded:
            bondDictionary[n] = []
            for j in np.concatenate((atomicArray[:n], atomicArray[n+1:]), axis=0):
                if x_boundary == "periodic":
                    if abs(i[0]-j[0]) < abs(min(i[0],j[0]) - max(i[0], j[0]) + xSize):
                        xDistance = j[0] - i[0]
                    elif i[0]<xSize/2:
                        xDistance = j[0] - xSize - i[0]
                    else:
                        xDistance = j[0] - i[0] + xSize
                else:
                    xDistance = j[0] - i[0]

                if y_boundary == "periodic":
                    if abs(i[1]-j[1]) < abs(min(i[1],j[1]) - max(i[1], j[1]) + ySize):
                        yDistance = j[1] - i[1]
                    elif i[1]<ySize/2:
                        yDistance = j[1] - ySize - i[1]
                    else:
                        yDistance = j[1] - i[1] + ySize
                else:
                    yDistance = j[1] - i[1]

                if z_boundary == "periodic":
                    if abs(i[2]-j[2]) < abs(min(i[2],j[2]) - max(i[2], j[2]) + zSize):
                        zDistance = j[2] - i[2]
                    elif i[2]<zSize/2:
                        zDistance = j[2] - zSize - i[2]
                    else:
                        zDistance = j[2] - i[2] + zSize
                else:
                    zDistance = j[2] - i[2]

                distance = math.sqrt(xDistance**2 + yDistance**2 + zDistance**2)
                if distance < bond_cutoff:
                    bondDictionary[n].append(np.array([xDistance, yDistance,zDistance]))
                
    #print(bondDictionary)
    for atom, bondVectors in bondDictionary.items():
        for i in bondVectors:
            length = np.linalg.norm(i)
            bondLengths.append(length)
        if len(bondVectors) < 3:
            print("2-fold coordinated atom ALERT")
        if len(bondVectors) < 4:
            danglingBonds+=1
        if len(bondVectors) > 4:
            floatingBonds += 1
        for n, i in enumerate(bondVectors[:-1]):
            for j in bondVectors[n+1:]:
                angle = np.arccos(np.dot(i,j)/(np.linalg.norm(i)*np.linalg.norm(j)))
                bondAngles.append(angle)

bondAngleArray = np.asarray(bondAngles)
bondLengthArray = np.array(bondLengths)
print("Mean bond length is: ",np.mean(bondLengthArray), " +- ",np.std(bondLengthArray))
print("Mean bond angle is: ",np.mean(bondAngleArray)*180/np.pi, " +- ",np.std(bondAngleArray)*180/np.pi)
print("There are ",danglingBonds/nfiles," dangling bonds per supercell.")
print("There are ",floatingBonds/nfiles," floating bonds per supercell.")
print("There are ",strainedBonds/nfiles," strained bonds per supercell.")

aSi_GAP_40_DFT.xyz
aSi_GAP_2_DFT.xyz
aSi_GAP_7_DFT.xyz
aSi_GAP_14_DFT.xyz
aSi_GAP_12_DFT.xyz
aSi_GAP_16_DFT.xyz
aSi_GAP_36_DFT.xyz
aSi_GAP_32_DFT.xyz
aSi_GAP_24_DFT.xyz
aSi_GAP_11_DFT.xyz
aSi_GAP_33_DFT.xyz
aSi_GAP_13_DFT.xyz
2-fold coordinated atom ALERT
aSi_GAP_42_DFT.xyz
aSi_GAP_1_DFT.xyz
aSi_GAP_10_DFT.xyz
aSi_GAP_38_DFT.xyz
aSi_GAP_21_DFT.xyz
aSi_GAP_35_DFT.xyz
aSi_GAP_3_DFT.xyz
aSi_GAP_27_DFT.xyz
aSi_GAP_17_DFT.xyz
aSi_GAP_18_DFT.xyz
aSi_GAP_23_DFT.xyz
aSi_GAP_45_DFT.xyz
aSi_GAP_47_DFT.xyz
aSi_GAP_44_DFT.xyz
2-fold coordinated atom ALERT
aSi_GAP_22_DFT.xyz
aSi_GAP_15_DFT.xyz
aSi_GAP_31_DFT.xyz
aSi_GAP_34_DFT.xyz
aSi_GAP_37_DFT.xyz
aSi_GAP_19_DFT.xyz
aSi_GAP_25_DFT.xyz
aSi_GAP_39_DFT.xyz
aSi_GAP_4_DFT.xyz
aSi_GAP_9_DFT.xyz
aSi_GAP_26_DFT.xyz
aSi_GAP_8_DFT.xyz
aSi_GAP_28_DFT.xyz
aSi_GAP_20_DFT.xyz
aSi_GAP_6_DFT.xyz
aSi_GAP_29_DFT.xyz
aSi_GAP_46_DFT.xyz
aSi_GAP_5_DFT.xyz
aSi_GAP_30_DFT.xyz
Mean bond length is:  2.3773374766908644  +-  0.043543966319751555
Mean bond angle is:  109.11